In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import JSONLoader
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.llms import Ollama

import os
import os.path as path
import json

In [ ]:
data_path = path.join("dataset", "corpus")
data_file_path = [path.join(data_path, file_name) for file_name in os.listdir(data_path)]

documents = []
for file_path in data_file_path:
  loader = JSONLoader(file_path, jq_schema='"Q: \(.question)\nA: \(.answer)"')
  documents.append(*loader.load())

page_content='Q: Methoxsalen(경구) 치료 후 환자가 장시간 햇빛에 노출될 경우 발생할 수 있는 가장 흔한 피부 관련 부작용은 무엇인가?
A: 광독성 반응' metadata={'source': '/Users/412ock/Projects/dl-trouble-detection/dataset/corpus/전문_622733.json', 'seq_num': 1}


In [16]:
embedding = HuggingFaceBgeEmbeddings(model_name="jhgan/ko-sroberta-multitask")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [58]:
vectorstore = FAISS.from_documents(documents, embedding)

In [60]:
retriever = vectorstore.as_retriever()

In [61]:
llm = Ollama(model="gemma3:4b", base_url="http://localhost:11434")

/var/folders/72/yqc7v5kd1rl4zp7rnrwkyx980000gn/T/ipykernel_46865/4044885202.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:4b", base_url="http://localhost:11434")


In [62]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [63]:
query = "발가락 사이에 피부 발진과 가려움이 동반되고 있어, 요즘에 수영장을 많이 갔는데 어떤 증상일까?"
response = qa_chain.run(query)
print("Response", response)

/var/folders/72/yqc7v5kd1rl4zp7rnrwkyx980000gn/T/ipykernel_46865/1112622304.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Response 이 환자의 증상은 발가락 사이의 발적, 가려움, 그리고 최근 수영장 방문 이력을 고려할 때, 무좀(interdigital tinea pedis)으로 진단할 수 있다. 치료로는 항진균제인 테르비나핀(terbinafine) 크림을 하루에 한 번 발진 부위와 그 주변의 건강한 피부에 얇게 바르는 방법이 적합하다.
